# Stage 6

## Running the experiments

In the previous stage, our team selected `Sammon's error` function and `Genetic algorithm` and `Simulated annealing` for the experiments, so we wil use them here.

### 1. Loading and splitting the data

In [579]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [580]:
# Loading the data
df = pd.read_csv('combined-metrics.csv')

In [581]:
X = df.drop('Name', axis=1).values
y = df['Name'].values

In [582]:
print(X.shape)

(101, 38)


In [583]:
X

array([[5.42625369e+00, 3.77802360e+01, 2.97935103e+01, ...,
        1.81400000e+03, 1.58500000e+04, 0.00000000e+00],
       [2.02732240e+00, 9.52868852e+00, 6.37568306e+00, ...,
        9.22000000e+02, 2.64500000e+03, 0.00000000e+00],
       [1.67987805e+00, 1.17048780e+01, 8.17073171e+00, ...,
        2.79100000e+03, 1.14790000e+04, 0.00000000e+00],
       ...,
       [4.29198473e+00, 2.41488550e+01, 1.87232824e+01, ...,
        3.82000000e+02, 6.43600000e+03, 0.00000000e+00],
       [9.15789474e+00, 3.77017544e+01, 2.96842105e+01, ...,
        1.78000000e+02, 1.38800000e+03, 0.00000000e+00],
       [2.24528302e+00, 1.04371069e+01, 7.09119497e+00, ...,
        8.01000000e+02, 2.98200000e+03, 0.00000000e+00]])

In [584]:
y

array(['ART', 'BentoML', 'Bokeh', 'Camel', 'CatBoost', 'Causal ML',
       'Chainer', 'Computer Vision', 'D2L', 'Darts', 'Dash', 'DeepChem',
       'DeepMind Control', 'DeepPavlov', 'Detectron2', 'DIGITS',
       'DragGan', 'EasyOCR', 'ELI5', 'EvalAI', 'facenet', 'FaceSwap',
       'Fairseq', 'FastAI', 'FeatureTools', 'FiftyOne', 'gensim',
       'Giskard', 'Gluonts', 'Google Flax', 'Google JAX', 'GPT-Engineer',
       'GPTDiscord', 'Gradio', 'Gymnasium', 'Horovod', 'ImageAI',
       'imbalanced-learn', 'InsightFace', 'Kaolin', 'Kedro', 'Keras',
       'Kserve', 'Lightning', 'Ludwig', 'Mage-ai', 'Mars', 'Matplotlib',
       'metatransformer', 'Mindsdb', 'MLflow', 'Mycroft',
       'Neural Prophet', 'NNI', 'Numpy', 'ONNX', 'Open-Assistant',
       'OpenAI Baselines', 'OpenAI Python API library', 'OpenVINO',
       'Optuna', 'Paddle', 'Pandas', 'Pocker', 'Pybrain', 'PyCaret',
       'pycm', 'PyMC', 'Pyro', 'PyTensor', 'PyTorch',
       'Pytorch image models', 'qlib', 'Rasa', 'Ray', 'Reco

In [585]:
# Splitting the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)

(80, 38)
(21, 38)


### 2. Determining the minimal subset of metrics on the training data

Sammon's error function that we will use as an objective

In [586]:
import numpy as np

# Sammon's error function
def sammons_error(X, Y):
    """
    X: Original high-dimensional data
    Y: Low-dimensional representation
    """
    # Calculate pairwise distances in X and Y
    dist_orig = np.sqrt(np.sum((X[:, np.newaxis, :] - X[np.newaxis, :, :]) ** 2, axis=-1))
    dist_lowd = np.sqrt(np.sum((Y[:, np.newaxis, :] - Y[np.newaxis, :, :]) ** 2, axis=-1))

    numerator = np.sum((dist_orig - dist_lowd) ** 2)
    denominator = np.sum(dist_orig ** 2)

    error = numerator / denominator

    return error

#### Genetic algorithm

In [587]:
def fitness_genetic(individual, X):
    # Create a subset of features based on the individual
    subset_X = X[:, np.array(individual).astype(bool)]

    # Calculate the fitness using the `sammons_error` function
    fitness = -sammons_error(X, subset_X)
    return fitness,

In [588]:
from deap import creator, base, tools, algorithms
import random
from sklearn.decomposition import PCA

# Set up the genetic algorithm
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


toolbox = base.Toolbox()


# Functions to limit the number of features in the individual to 25
def create_individual():
    individual = [random.randint(0, 1) for _ in range(X.shape[1])]
    while sum(individual) > 25:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(individual) - 25, replace=False)
        for idx in indices:
            individual[idx] = 0
    return creator.Individual(individual)


def mate(ind1, ind2):
    child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
    tools.cxTwoPoint(child1, child2)

    # Limit the number of features in the child to 25
    while sum(child1) > 25:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(child1) - 25, replace=False)
        for idx in indices:
            child1[idx] = 0
    while sum(child2) > 25:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(child2) - 25, replace=False)
        for idx in indices:
            child2[idx] = 0
    return child1, child2


def mutate(individual):
    child = toolbox.clone(individual)
    tools.mutFlipBit(child, indpb=0.05)
    while sum(child) > 25:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(child) - 25, replace=False)
        for idx in indices:
            child[idx] = 0
    return child,


toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_genetic, X=X_train)
toolbox.register("mate", mate)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

d:\Projects giperion\lsd-metrics\venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
d:\Projects giperion\lsd-metrics\venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [589]:
import numpy as np

# Run the genetic algorithm
population = toolbox.population(n=100)
hof = tools.HallOfFame(100)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

population, log = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=10000, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg      	std     	min      	max         
0  	100   	-0.318637	0.289892	-0.878609	-1.86772e-06
1  	65    	-0.129461	0.197598	-0.606732	-3.97952e-06


2  	59    	-0.0314185	0.106461	-0.565846	-3.97952e-06
3  	56    	-0.0183246	0.0906814	-0.534266	-3.97952e-06
4  	72    	-0.000360199	0.00154778	-0.0155907	-3.93296e-07
5  	58    	-0.000674441	0.00477525	-0.0456176	-3.86481e-07
6  	55    	-0.0165837  	0.090843  	-0.533198 	-3.86481e-07
7  	66    	-0.00137938 	0.00670093	-0.0449251	-1.96827e-09
8  	60    	-0.000464704	0.00446321	-0.0448676	-4.48024e-10
9  	53    	-0.000450825	0.0043426 	-0.0436534	-4.48024e-10
10 	63    	-0.00622235 	0.0532677 	-0.532709 	-4.48024e-10
11 	62    	-0.00046047 	0.00254845	-0.0149583	-4.48024e-10
12 	63    	-0.00650931 	0.053267  	-0.532632 	-2.81741e-12
13 	61    	-0.0174713  	0.0906815 	-0.532735 	-2.81741e-12
14 	58    	-0.0223056  	0.104176  	-0.53263  	-2.51064e-12
15 	67    	-0.0063033  	0.055319  	-0.554648 	-2.51064e-12
16 	55    	-0.0164245  	0.0906606 	-0.531426 	-2.48929e-12
17 	56    	-0.0214246  	0.104148  	-0.532838 	-2.48929e-12
18 	54    	-0.0165345  	0.0906233 	-0.531279 	-6.32633e-13
19 	56

In [591]:
# Get the best individual (subset of metrics)
best_individual = hof.items[0]
best_subset_genetic = [df.columns[i+1] for i, bit in enumerate(best_individual) if bit]

print("Best subset of metrics:", best_subset_genetic)
print("Best fitness:", fitness_genetic(best_individual, X_train))
print("Length of best subset:", len(best_subset_genetic))

Best subset of metrics: ['h2', 'N2', 'vocabulary', 'length', 'volume', 'effort', 'time', 'CCL', 'CCO', 'CEE', 'CI', 'LDC', 'LLDC', 'TLLOC', 'TLOC', 'TNCL', 'TNFI', 'TNM', 'TNOS', 'TNPKG', 'TCLOC', 'TNDI', 'WarningCritical', 'WarningMajor', 'WarningMinor']
Best fitness: (-2.8335872964719968e-16,)
Length of best subset: 25


#### Simulated annealing

In [595]:
import math

def fitness_annealing(X, subset_X):
    return -sammons_error(X, subset_X)

def accept_probability(curr_score, best_score, temperature):
    if curr_score > best_score:
        return 1.0
    else:
        return math.exp((curr_score - best_score) / temperature)
    
def generate_random_subset(X):
    # Generate new random subset
    subset = np.random.randint(2, size=(38,)) # 0 or 1 masks

    # Limit the number of features in the subset to 25
    while sum(subset) > 25:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(subset) - 25, replace=False)
        for idx in indices:
            subset[idx] = 0

    return subset

def annealing(X):
    # Simulated annealing parameters
    t = 100000 # Initial temperature
    t_min = 1e-3 # Minimum temperature
    cooling_factor = 0.99999 # Temperature damping factor
    max_iterations = 100000 # Maximum number of iterations
    best_subset = None
    best_score = float("-inf")
    
    for i in range(max_iterations):

        subset = generate_random_subset(X) # Generate random subset    

        score = fitness_annealing(X, X[:,subset==1]) # Evaluate subset

        # Update best, higher is better
        if score > best_score or random.random() < accept_probability(score, best_score, T):
            best_subset = subset
            best_score = score

        # Cool temperature
        t = t * cooling_factor

        # Check if cooled enough
        if t < t_min:
            break

    return best_subset, best_score

best_subset, best_score = annealing(X_train)

print("Best subset:", best_subset)
print("Best sammon's error result:", -best_score)

Best subset: [1 1 0 0 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 1 0 0 1 1 1 1 1 1
 1]
Best sammon's error result: 0.0004110620517737251


In [596]:
best_subset_annealing = [df.columns[i+1] for i, bit in enumerate(best_subset) if bit]
print(best_subset_annealing)
print(len(best_subset_annealing))

['h1', 'h2', 'length', 'difficulty', 'effort', 'bugs', 'CEE', 'CI', 'LDC', 'CLLC', 'CLC', 'NCR', 'CEG', 'TLLOC', 'TLOC', 'TNM', 'TCLOC', 'TNDI', 'WarningBlocker', 'WarningCritical', 'WarningMajor', 'WarningMinor', 'WarningInfo']
23


In [599]:
print(best_subset_annealing)
print(best_subset_genetic)


print(list(best_individual))
print(list(best_subset))


X_genetic = X_train[:, np.array(best_individual).astype(bool)]
X_annealing = X_train[:, np.array(best_subset).astype(bool)]

X_genetic_test = X_test[:, np.array(best_individual).astype(bool)]
X_annealing_test = X_test[:, np.array(best_subset).astype(bool)]

print("Genetic training score:", sammons_error(X_train, X_genetic))
print("Annealing training score:", sammons_error(X_train, X_annealing))

print("Genetic validation score:", sammons_error(X_test, X_genetic_test))
print("Annealing validation score:", sammons_error(X_test, X_annealing_test))

['h1', 'h2', 'length', 'difficulty', 'effort', 'bugs', 'CEE', 'CI', 'LDC', 'CLLC', 'CLC', 'NCR', 'CEG', 'TLLOC', 'TLOC', 'TNM', 'TCLOC', 'TNDI', 'WarningBlocker', 'WarningCritical', 'WarningMajor', 'WarningMinor', 'WarningInfo']
['h2', 'N2', 'vocabulary', 'length', 'volume', 'effort', 'time', 'CCL', 'CCO', 'CEE', 'CI', 'LDC', 'LLDC', 'TLLOC', 'TLOC', 'TNCL', 'TNFI', 'TNM', 'TNOS', 'TNPKG', 'TCLOC', 'TNDI', 'WarningCritical', 'WarningMajor', 'WarningMinor']
[0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0]
[1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]
Genetic training score: 2.8335872964719968e-16
Annealing training score: 0.0004110620517737251
Genetic validation score: 1.0593753465106679e-16
Annealing validation score: 0.00011898381991976522
